In [1]:
# Imports
import pandas as pd
import numpy as np
from IPython.display import HTML, display
import warnings
import os
import time
from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV, LassoLarsCV, RidgeCV
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from xgboost import XGBRegressor, DMatrix

# Visualization options
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', None)
%matplotlib inline
display(HTML("<style>.container { width: 90% !important; }</style>"))
warnings.filterwarnings("ignore")

# Global variables
SEED = 420
N_JOBS = 18
FOLDS = 10

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


<b>Functions</b>

In [2]:
# Stack all single Level 1 models OOF and test preds as entries for a Level 2 model
def get_level_one_data(oof_preds_dir, oof_targets_dir, preds_dir) :
    level_one_preds = pd.DataFrame()
    i = 0
    for file_name in sorted(os.listdir(oof_preds_dir)) :
        preds_OOF = pd.read_csv(oof_preds_dir + file_name)
        cur_preds_OOF = pd.Series(preds_OOF.y, name = file_name[0:7])
        level_one_preds = pd.concat([level_one_preds, cur_preds_OOF], axis = 1)
        i += 1

    level_one_targets = pd.DataFrame()
    i = 0
    for file_name in sorted(os.listdir(oof_targets_dir)) :
        targets_OOF = pd.read_csv(oof_targets_dir + file_name)
        cur_targets_OOF = pd.Series(targets_OOF.y, name = file_name[0:7])
        level_one_targets = pd.concat([level_one_targets, cur_targets_OOF], axis = 1)
        i += 1

    # Same for test set preds
    level_one_test = pd.DataFrame()
    i = 0
    for file_name in sorted(os.listdir(preds_dir)) :
        preds_test = pd.read_csv(preds_dir + file_name)
        cur_preds_test = pd.Series(preds_test.y, name = file_name[0:7])
        level_one_test = pd.concat([level_one_test, cur_preds_test], axis = 1)
        i += 1
    
    return(level_one_preds, level_one_targets, level_one_test)

In [3]:
def get_cv_score(model, X_train, X_train_y) :
    oof_preds = pd.DataFrame()
    oof_targets = pd.DataFrame()
    kf = KFold(n_splits = FOLDS, shuffle = True, random_state = SEED)
    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        #print("FOLD " + str(i + 1))
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = X_train_y.iloc[train_index], X_train_y.iloc[test_index]

        # Fit and predict
        model.fit(X_train_fold, y_train_fold)
        preds_X_test_fold = model.predict(X_test_fold)
        
        # Assemble OOF predictions and targets
        oof_preds = pd.concat([pd.Series(oof_preds), pd.Series(preds_X_test_fold)], axis = 0)
        oof_targets = pd.concat([pd.Series(oof_targets), pd.Series(y_test_fold)], axis = 0)
        
    # Compute error on concatenated OOF predictions
    cv_score = r2_score(oof_targets, oof_preds)
    print("Global OOF r2_score : " + str(cv_score))
                    
    return(cv_score)

In [4]:
def replace_probed_y(row):
    if (np.isnan(row["y_y"])) :
        return (row["y_x"])
    else :
        return (row["y_y"])

<b>Models</b>

In [5]:
models = {
    "la" : LassoCV(eps = 0.0001, 
                   n_alphas = 100, 
                   max_iter = 10000, 
                   tol = 0.0001,                     
                   normalize = True, 
                   precompute = True, 
                   random_state = SEED,
                   n_jobs = N_JOBS),
    "ll" : LassoLarsCV(max_n_alphas = 1000, 
                       max_iter = 10000,
                       normalize = True, 
                       precompute = True, 
                       n_jobs = N_JOBS),
    "ri" : RidgeCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1, 3, 6],
                   normalize = True),
    "rf" : RandomForestRegressor(n_estimators = 200,
                                 criterion = "mse", 
                                 max_features = 0.75,
                                 max_depth = 8,
                                 min_samples_split = 5,
                                 min_samples_leaf = 2, 
                                 bootstrap = True, 
                                 n_jobs = N_JOBS, 
                                 random_state = SEED),
    "et" : ExtraTreesRegressor(n_estimators = 200,
                               criterion = "mse", 
                               max_features = 0.75,
                               max_depth = 8,
                               min_samples_split = 5,
                               min_samples_leaf = 2, 
                               bootstrap = True, 
                               n_jobs = N_JOBS, 
                               random_state = SEED), 
    "kn" : KNeighborsRegressor(n_neighbors = 100, 
                               weights = "distance", 
                               p = 3, 
                               n_jobs = N_JOBS), 
    "xg" : XGBRegressor(objective = "reg:linear", 
                        learning_rate = 0.03, 
                        n_estimators = 200, 
                        max_depth = 8,
                        min_child_weight = 2, 
                        subsample = 0.75, 
                        colsample_bytree = 0.75, 
                        colsample_bylevel = 0.75, 
                        nthread = N_JOBS,
                        seed = SEED)
#    "gb" : GradientBoostingRegressor(loss = "ls", 
#                                     learning_rate = 0.03, 
#                                     n_estimators = 200, 
#                                     max_depth = 8,
#                                     criterion = "friedman_mse",
#                                     min_samples_split = 5,
#                                     min_samples_leaf = 2,
#                                     subsample = 0.75,
#                                     max_features = 0.75, 
#                                     random_state = SEED),
}

<b>Script</b>

In [6]:
# Get data
init_test = pd.read_csv("raw_data/test.csv")
test_ids = init_test.ID.values


In [7]:
# Define probe df
probe_ids = [1, 12, 23, 28, 42, 
             43, 45, 57, 72, 78, 
             88, 89, 93, 94, 104, 
             105, 110, 253, 259, 262, 
             289, 337, 409, 437, 488, 
             493, 973, 1001, 1004, 1008, 
             1009, 1259, 1644, 1652, 1664, 
             2129, 2342, 3853, 3977, 4193, 
             4958, 4960, 7055, 7805, 8002, 
             8007, 8416]
probe_values = [71.34112, 109.30903, 115.21953, 92.00675, 87.73572, 
                129.79876, 99.55671, 116.02167, 110.54742, 125.28849, 
                90.33211, 130.55165, 105.79792, 103.04672, 92.37968, 
                108.5069, 87.70757, 115.93724, 93.33662, 75.35182, 
                89.27667, 101.23135, 91.00760, 85.96960, 113.39009, 
                108.40135, 106.76189, 111.65212, 91.472, 106.71967, 
                108.21841, 112.3909, 99.14157, 89.77625, 112.93977, 
                112.03, 93.06, 105.481283411, 132.08556, 132.78216, 
                113.58711, 89.83957, 91.549, 105.8472, 95.84858, 
                87.44019, 96.84773]
new_train = init_test[init_test["ID"].isin(probe_ids)]
new_y = pd.DataFrame({"y" : probe_values})
new_y.set_index(new_train.index, inplace = True)
probe_df = pd.concat([new_train, new_y], axis = 1)


In [8]:
# Prepare process
oof_preds_dirs = []
oof_targets_dirs = []
preds_dirs = []

oof_preds_dirs.append("oof_preds/dc1_4/")
oof_preds_dirs.append("oof_preds/dc5_8/")
oof_preds_dirs.append("oof_preds/dc9_12/")
oof_preds_dirs.append("oof_preds/dc13_16/")

oof_targets_dirs.append("oof_targets/dc1_4/")
oof_targets_dirs.append("oof_targets/dc5_8/")
oof_targets_dirs.append("oof_targets/dc9_12/")
oof_targets_dirs.append("oof_targets/dc13_16/")

preds_dirs.append("preds/dc1_4/")
preds_dirs.append("preds/dc5_8/")
preds_dirs.append("preds/dc9_12/")
preds_dirs.append("preds/dc13_16/")


In [9]:
# Run stacker
for i in range(len(oof_preds_dirs)) :
    print("********************")
    oof_preds_dir = oof_preds_dirs[i]
    oof_targets_dir = oof_targets_dirs[i]
    preds_dir = preds_dirs[i]

    # Create Level 2 inputs
    level_one_preds, level_one_targets, level_one_test = get_level_one_data(oof_preds_dir, oof_targets_dir, preds_dir)
    display(level_one_preds.head(2))
    print(level_one_preds.shape)
    
    # Get error of each OOF column
    for col in level_one_preds.columns :
        cv_score = r2_score(level_one_targets[col], level_one_preds[col])
        print("Global OOF r2_score for " + col + " : " + str(cv_score))

    # Try different level 2 models
    for model_name, model  in models.items() :
        print("***** MODEL : " + model_name + " *****")

        # Get CV score
        cv_score = get_cv_score(model, level_one_preds, level_one_targets.iloc[:, 0])

        # Fit, predict
        model.fit(level_one_preds, level_one_targets.iloc[:, 0])
        preds = model.predict(level_one_test)
        
        # Handle format problems
        #if isinstance(model, (RidgeCV, KNeighborsRegressor)) :
        #    preds = pd.DataFrame(preds).iloc[:, 0].values            
                        
        # Modify probed values
        conf_name = oof_preds_dirs[i].split("/")[1]
        file_name = "ens_preds/stacker_" + conf_name + "_" + model_name + ".csv"
        final_preds = pd.DataFrame({"ID": init_test.ID.values, "y": pd.DataFrame(preds)[0]})
        final_preds = pd.merge(final_preds, probe_df[["ID", "y"]], how = "left", on = "ID")
        final_preds["y"] = final_preds.apply(replace_probed_y, axis = 1)
        final_preds.drop(["y_x", "y_y"], axis = 1, inplace = True)

        # Save predictions
        final_preds.to_csv(file_name, index = False)
        display(final_preds.head(2))


********************


,et_dc1_,et_dc2_,et_dc3_,et_dc4_,la_dc1_,la_dc2_,la_dc3_,la_dc4_,ll_dc1_,ll_dc2_,ll_dc3_,ll_dc4_,rf_dc1_,rf_dc2_,rf_dc3_,rf_dc4_,xg_dc1_,xg_dc2_,xg_dc3_,xg_dc4_,ze_dc1_,ze_dc2_,ze_dc3_,ze_dc4_,zl_dc1_,zl_dc2_,zl_dc3_,zl_dc4_,zr_dc1_,zr_dc2_,zr_dc3_,zr_dc4_,zx_dc1_,zx_dc2_,zx_dc3_,zx_dc4_
0,116.13408,114.13028,116.18572,114.25545,115.39651,110.61552,115.39649,110.47323,115.42086,108.01324,115.43921,104.70043,114.68859,113.86919,117.25145,114.41533,115.41705,113.37335,119.43770,115.68615,114.52726,115.31362,114.53231,115.18735,115.70149,114.29528,116.25454,114.70613,114.47179,115.38637,115.25877,115.10238,114.87389,112.75110,116.03893,113.86649
1,113.33418,112.93224,113.40276,113.08442,114.19929,110.66778,114.19928,110.47323,114.20741,108.02133,114.21384,104.70043,112.07628,113.14277,112.31983,113.07338,111.84063,111.02529,111.50563,112.02895,114.61618,115.11454,114.58866,115.08703,114.22569,113.50603,114.40470,113.37642,112.36361,115.33690,112.84508,114.96566,112.05869,111.37788,112.27528,111.95670


(3953, 36)
Global OOF r2_score for et_dc1_ : 0.572600357478
Global OOF r2_score for et_dc2_ : 0.56949721826
Global OOF r2_score for et_dc3_ : 0.572999418005
Global OOF r2_score for et_dc4_ : 0.569112855322
Global OOF r2_score for la_dc1_ : 0.588467268349
Global OOF r2_score for la_dc2_ : 0.548622901878
Global OOF r2_score for la_dc3_ : 0.588467136131
Global OOF r2_score for la_dc4_ : 0.550172023209
Global OOF r2_score for ll_dc1_ : 0.588498942363
Global OOF r2_score for ll_dc2_ : 0.53041351412
Global OOF r2_score for ll_dc3_ : 0.588555948425
Global OOF r2_score for ll_dc4_ : 0.522219953052
Global OOF r2_score for rf_dc1_ : 0.572124235188
Global OOF r2_score for rf_dc2_ : 0.567171849033
Global OOF r2_score for rf_dc3_ : 0.574580627741
Global OOF r2_score for rf_dc4_ : 0.568533648688
Global OOF r2_score for xg_dc1_ : 0.564998266541
Global OOF r2_score for xg_dc2_ : 0.564896677099
Global OOF r2_score for xg_dc3_ : 0.565711036022
Global OOF r2_score for xg_dc4_ : 0.565157028005
Global OOF 

,ID,y
0,1,71.34112
1,2,94.55080


***** MODEL : la *****
Global OOF r2_score : 0.588623997412


...

,ID,y
0,1,71.34112
1,2,94.54255


***** MODEL : et *****
Global OOF r2_score : 0.580195546938


,ID,y
0,1,71.34112
1,2,94.44497


***** MODEL : ll *****
Global OOF r2_score : 0.583793090577


,ID,y
0,1,71.34112
1,2,84.40949
